In [ ]:
# Atualização das bibliotecas
!pip install --upgrade keras tensorflow

In [ ]:
# Download do dataset de Treinamento
!gdown https://drive.google.com/uc?id=1xQqTSUA-celn_KRB9Qpx77g54KR0WHaH
!unzip Leishmania_Treinamento.zip
!rm Leishmania_Treinamento.zip

In [ ]:
# Fixando seeds pra ter resultados mais consistentes
import numpy as np
import tensorflow as tf
import random as python_random

np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

In [ ]:
# Imports
from os import listdir
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Inicialização do modelo
classificador = Sequential()

tamanho_imagem = (224, 224)
# Carregamento da arquitetura
transfer = ResNet152(include_top = False,
                     input_shape = (tamanho_imagem[0], 
                                    tamanho_imagem[1], 3))
for camada in transfer.layers:
  camada.trainable = True

classificador.add(transfer)

# Saída do modelo
classificador.add(Flatten())
classificador.add(Dense(1, activation = 'sigmoid'))

classificador.compile(optimizer = RMSprop(lr = 0.00004),
               loss = 'binary_crossentropy', metrics = ['accuracy'])

batch = 3

# Nenhuma forma de Augmentation
# Rescale é apenas normalização
datagen = ImageDataGenerator(rescale=1./255)


grupo_treinamento = datagen.flow_from_directory('Leishmania_Treinamento/treinamento', 
                                                target_size=tamanho_imagem, 
                                                batch_size=batch, 
                                                class_mode='binary')

grupo_validacao = datagen.flow_from_directory('Leishmania_Treinamento/validacao', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')

classificador.fit(grupo_treinamento, epochs=15, validation_data=grupo_validacao)

Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/15
134/134 [==============================] - 30s 223ms/step - loss: 0.2783 - accuracy: 0.8875 - val_loss: 72.5824 - val_accuracy: 0.5000
Epoch 2/15
134/134 [==============================] - 28s 208ms/step - loss: 0.0880 - accuracy: 0.9675 - val_loss: 380.2211 - val_accuracy: 0.5000
Epoch 3/15
134/134 [==============================] - 27s 205ms/step - loss: 0.0275 - accuracy: 0.9850 - val_loss: 359.8334 - val_accuracy: 0.5000
Epoch 4/15
134/134 [==============================] - 27s 205ms/step - loss: 0.0023 - accuracy: 0.9975 - val_loss: 278.0437 - val_accuracy: 0.5000
Epoch 5/15
134/134 [==============================] - 28s 205ms/step - loss: 0.0040 - accuracy: 0.9975 - val_loss: 237.8790 - val_accuracy: 0.5000
Epoch 6/15
134/134 [==============================] - 28s 206ms/step - loss: 2.0318e-05 - accuracy: 1.0000 - val_loss: 133.0392 - val_accuracy: 0.5000
Epoch 7/15
134/134 [=============

In [ ]:
# Download do dataset de Teste
!gdown https://drive.google.com/uc?id=1tTBldPLMQk8jLLDjBaxwbl_FXpYNS0Ux
!unzip Leishmania_Teste.zip
!rm Leishmania_Teste.zip

In [ ]:
# Funções

# Preprocessamento das imagens
def carregar_imagem(arquivo):
  img = image.load_img(arquivo, target_size=tamanho_imagem)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = img/255

  return img

# Carregar imagens em uma lista
def predicao_teste(pasta_teste):
  imagens = []
  for pasta in list(reversed(listdir(pasta_teste))):
    for arquivo in listdir(pasta_teste + '/' + pasta):
      imagens.append(carregar_imagem(pasta_teste + pasta + '/' + arquivo))
  
  return imagens

In [ ]:
imagens = np.vstack(predicao_teste('Leishmania_Teste/'))
respostas_modelo = classificador.predict_classes(imagens)
respostas_teste = datagen.flow_from_directory('Leishmania_Teste/', 
                                              target_size=tamanho_imagem, 
                                              batch_size=batch, 
                                              class_mode='binary')
print('Respostas do modelo\n' + str(np.concatenate(respostas_modelo)))
print('Respostas reais\n' + str(respostas_teste.classes))

# Matriz de confusão
matriz_de_confusao = confusion_matrix(respostas_teste.classes, respostas_modelo)

# Metricas
TrueNegative, FalsePositive, FalseNegative, TruePositive = matriz_de_confusao.ravel()

# Precisão
print('Precisao ' + str((TruePositive + TrueNegative) / len(respostas_modelo)))

# Sensibilidade
print('Sensibilidade ' + str(TruePositive / (TruePositive + FalseNegative)))

# Especificidade
print('Especificidade ' + str(TrueNegative / (FalsePositive + TrueNegative)))

In [ ]:
# Salvar modelo treinado em um arquivo
classificador.save('modelo_leishmania.h5')